In [1]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveAll.p", "rb" ) )


In [2]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,17,3.0
1,0,29,153.0
2,0,42,7.0
3,0,46,518.0
4,0,47,5.0


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [4]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>25)
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)
ratings = ratingsPD2

In [5]:
len(ratings)

27265784

In [6]:
ratings['item'].nunique()

428623

In [7]:
ratings['user'].nunique()

42345

In [13]:
ratingsPD.groupby('item').count().sort_values(by=['user'], ascending=False).head(50)

,user,rating
item,,
1465,19022,19022
45,18769,18769
313,18439,18439
132,17488,17488
26,16789,16789
150,16643,16643
137,16536,16536
234,16110,16110
15,16071,16071


In [10]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

rm = 0

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    pred = als.BiasedMF(45)
    rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))

0
0.33308603563796746
0
0.33090071233699403
0
0.33492574233861877
4
0.3345198537603395
0
0.3319590372805881
Average RMSE: 0.3330782762709016


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    pred = als.BiasedMF(20)
    base = Bias(damping=5)
    #rec = Recommender.adapt(pred)
    fall = Fallback(pred,base)
    fall.fit(train)
    preds = batch.predict(fall, test)
    print(preds.head())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
